In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("car.csv")
df.head()

,Age,Have_Car,Have_Insurance
0,21,0,0
1,48,1,1
2,28,0,0
3,19,0,0
4,56,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['Age','Have_Insurance']],df.Have_Car, test_size=0.2, random_state=25)

In [4]:
X_train[:10]

,Age,Have_Insurance
0,21,0
13,72,1
6,32,0
17,28,0
24,25,0
19,33,0
25,30,1
16,22,0
20,36,0
3,19,0


In [5]:
df.shape

(27, 3)

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled['Age'] = X_train_scaled['Age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['Age'] = X_test_scaled['Age'] / 100

In [7]:
X_train_scaled[:10]

,Age,Have_Insurance
0,0.21,0
13,0.72,1
6,0.32,0
17,0.28,0
24,0.25,0
19,0.33,0
25,0.30,1
16,0.22,0
20,0.36,0
3,0.19,0


In [14]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='mse',
              #loss='binary_crossentropy', # Try binary_crossentropy
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
1/1 [==============================] - 1s 573ms/step - loss: 0.2015 - accuracy: 0.6190
Epoch 2/500
1/1 [==============================] - 0s 13ms/step - loss: 0.2015 - accuracy: 0.6190
Epoch 3/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2014 - accuracy: 0.6190
Epoch 4/500
1/1 [==============================] - 0s 12ms/step - loss: 0.2014 - accuracy: 0.6190
Epoch 5/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2013 - accuracy: 0.6190
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 0.2013 - accuracy: 0.6190
Epoch 7/500
1/1 [==============================] - 0s 16ms/step - loss: 0.2012 - accuracy: 0.6190
Epoch 8/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2012 - accuracy: 0.6190
Epoch 9/500
1/1 [==============================] - 0s 11ms/step - loss: 0.2012 - accuracy: 0.6190
Epoch 10/500
1/1 [==============================] - 0s 16ms/step - loss: 0.2011 - accuracy: 0.6190
Epoch 11/500
1/1 [

In [15]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 112ms/step - loss: 0.5000 - accuracy: 0.5000


[0.5, 0.5]

In [16]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 241ms/step - loss: 0.1192 - accuracy: 1.0000


[0.11922436952590942, 1.0]

In [17]:
X_test_scaled

,Age,Have_Insurance
2,0.28,0
10,0.53,1
21,0.56,1
11,0.33,0
14,0.80,1
9,0.22,0


In [18]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 76ms/step


array([[0.4581661 ],
       [0.81906116],
       [0.82256013],
       [0.46802443],
       [0.84865785],
       [0.44637984]], dtype=float32)

In [19]:
y_test

2     0
10    1
21    1
11    0
14    1
9     0
Name: Have_Car, dtype: int64

Let's get the values of w and b.

In [20]:
w, b = model.get_weights()
w, b

(array([[0.7930133],
        [1.479474 ]], dtype=float32),
 array([-0.38977152], dtype=float32))

Code from scratch

In [23]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))
sigmoid(10)

0.9999546021312976

In [24]:
def prediction_function(age, have_insurance):
    weighted_sum = w[0]*age + w[1]*have_insurance + b
    return sigmoid(weighted_sum)

In [25]:
prediction_function(.53, 1)

0.8190611347672608

Choose either log_loss or mse.

In [26]:
delta = 1e-15
def log_loss(true_value, predicted_value):
  predicted_new = [max(i, delta) for i in predicted_value]
  predicted_new = [min(i, 1-delta) for i in predicted_new]
  predicted_new = np.array(predicted_new)
  return -np.mean(true_value*np.log(predicted_new)+(1-true_value)*np.log(1-predicted_new))

In [27]:
def mse(true_value, predicted_value):
  return np.mean(np.square(true_value - predicted_value))

In [28]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

In [29]:
sigmoid_numpy(np.array([10,0,1]))

array([0.9999546 , 0.5       , 0.73105858])

In [30]:
def gradient_descent(age, have_insurance, y_true, epochs):
  w1 = w2 = 1
  bias = 0
  lr = 0.01
  n = len(age)

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * have_insurance + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = mse(y_true, y_predicted) # log_loss instead of "mse" if you want to use log_loss

    w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
    w2d = (1/n) * np.dot(np.transpose(have_insurance), (y_predicted - y_true))

    biasd = np.mean(y_predicted - y_true)

    w1 = w1 - lr * w1d
    w2 = w2 - lr * w2d
    bias = bias - lr * biasd

    print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
  return w1, w2, bias

In [31]:
gradient_descent(X_train_scaled['Age'], X_train_scaled['Have_Insurance'], y_train, 500)

Epoch:0, w1:0.9998014060105598, w2:0.9999772002787576, bias:-0.0008608535754804835, loss:0.20154142950844003
Epoch:1, w1:0.9996035393198373, w2:0.9999551525531023, bias:-0.0017198854902701296, loss:0.2015040130316338
Epoch:2, w1:0.9994063983128003, w2:0.9999338550493376, bias:-0.0025770997405222843, loss:0.20146674466527476
Epoch:3, w1:0.9992099813759415, w2:0.9999133059946234, bias:-0.0034325003177399856, loss:0.20142962381187934
Epoch:4, w1:0.9990142868972903, w2:0.9998935036169939, bias:-0.004286091208752697, loss:0.20139264987565914
Epoch:5, w1:0.9988193132664246, w2:0.9998744461453775, bias:-0.005137876395693263, loss:0.201355822262524
Epoch:6, w1:0.9986250588744822, w2:0.9998561318096142, bias:-0.005987859855975077, loss:0.20131914038008344
Epoch:7, w1:0.9984315221141725, w2:0.9998385588404746, bias:-0.006836045562269465, loss:0.20128260363764988
Epoch:8, w1:0.9982387013797881, w2:0.9998217254696778, bias:-0.0076824374824832845, loss:0.2012462114462404
Epoch:9, w1:0.9980465950672

(0.9634079522525882, 1.0506180021038023, -0.2713934858831498)

In [32]:
w, b

(array([[0.7930133],
        [1.479474 ]], dtype=float32),
 array([-0.38977152], dtype=float32))